In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    return actions_logits, aloss, dloss, aloss2, (1-dones)

In [8]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2, self.dones__ = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [10]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [11]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [13]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [15]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            idx = np.random.choice(np.arange(memory_size// batch_size))
            states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            states = states[rates == np.max(rates)]
            actions = actions[rates == np.max(rates)]
            next_states = next_states[rates == np.max(rates)]
            rewards = rewards[rates == np.max(rates)]
            dones = dones[rates == np.max(rates)]
            rates = rates[rates == np.max(rates)]
            aloss, dloss, _, _ = sess.run([model.a_loss, model.d_loss, model.a_opt, model.d_opt],
                                          feed_dict = {model.states: states, 
                                                       model.actions: actions,
                                                       model.next_states: next_states,
                                                       model.rewards: rewards,
                                                       model.dones: dones,
                                                       model.rates: rates})
            aloss2, _, dones__ = sess.run([model.a_loss2, model.a_opt2, model.dones__], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            #print(dones__, np.max(rates), rates.shape, dones)
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:14.0000 R:14.0000 rate:0.0280 aloss:1.4010 dloss:1.4333 aloss2:1.3399 exploreP:0.9986
Episode:1 meanR:23.0000 R:32.0000 rate:0.0640 aloss:1.3975 dloss:1.4208 aloss2:1.3527 exploreP:0.9955
Episode:2 meanR:21.3333 R:18.0000 rate:0.0360 aloss:1.3961 dloss:1.4052 aloss2:1.3666 exploreP:0.9937
Episode:3 meanR:20.2500 R:17.0000 rate:0.0340 aloss:1.3865 dloss:1.3843 aloss2:1.3897 exploreP:0.9920
Episode:4 meanR:18.4000 R:11.0000 rate:0.0220 aloss:1.3805 dloss:1.3776 aloss2:1.3967 exploreP:0.9909
Episode:5 meanR:20.8333 R:33.0000 rate:0.0660 aloss:1.3899 dloss:1.3719 aloss2:1.3987 exploreP:0.9877
Episode:6 meanR:20.4286 R:18.0000 rate:0.0360 aloss:1.3877 dloss:1.3559 aloss2:1.4124 exploreP:0.9859
Episode:7 meanR:19.7500 R:15.0000 rate:0.0300 aloss:1.3890 dloss:1.3542 aloss2:1.4158 exploreP:0.9845
Episode:8 meanR:23.8889 R:57.0000 rate:0.1140 aloss:1.4100 dloss:1.3427 aloss2:1.4213 exploreP:0.9789
Episode:9 meanR:23.1000 R:16.0000 rate:0.0320 aloss:1.4037 dloss:1.3146 aloss2:1.4

Episode:80 meanR:22.5802 R:17.0000 rate:0.0340 aloss:1.5245 dloss:0.7304 aloss2:2.7751 exploreP:0.8345
Episode:81 meanR:22.5854 R:23.0000 rate:0.0460 aloss:1.5622 dloss:0.7420 aloss2:2.8884 exploreP:0.8326
Episode:82 meanR:22.6988 R:32.0000 rate:0.0640 aloss:1.5906 dloss:0.6995 aloss2:2.9667 exploreP:0.8300
Episode:83 meanR:22.5714 R:12.0000 rate:0.0240 aloss:1.5780 dloss:0.7058 aloss2:2.9778 exploreP:0.8290
Episode:84 meanR:22.4941 R:16.0000 rate:0.0320 aloss:1.5858 dloss:0.6776 aloss2:2.9940 exploreP:0.8277
Episode:85 meanR:22.6047 R:32.0000 rate:0.0640 aloss:1.5757 dloss:0.6783 aloss2:3.0549 exploreP:0.8251
Episode:86 meanR:22.4828 R:12.0000 rate:0.0240 aloss:1.5649 dloss:0.6916 aloss2:3.0110 exploreP:0.8241
Episode:87 meanR:22.3636 R:12.0000 rate:0.0240 aloss:1.5290 dloss:0.7076 aloss2:3.0174 exploreP:0.8231
Episode:88 meanR:22.2472 R:12.0000 rate:0.0240 aloss:1.5477 dloss:0.6807 aloss2:3.1045 exploreP:0.8222
Episode:89 meanR:22.4444 R:40.0000 rate:0.0800 aloss:1.5688 dloss:0.6831 

Episode:159 meanR:19.3600 R:13.0000 rate:0.0260 aloss:1.4074 dloss:0.5151 aloss2:5.0368 exploreP:0.7175
Episode:160 meanR:19.5300 R:28.0000 rate:0.0560 aloss:1.4095 dloss:0.4741 aloss2:5.0657 exploreP:0.7156
Episode:161 meanR:19.4500 R:13.0000 rate:0.0260 aloss:1.4049 dloss:0.5152 aloss2:5.1125 exploreP:0.7147
Episode:162 meanR:19.4300 R:12.0000 rate:0.0240 aloss:1.3988 dloss:0.4702 aloss2:5.0313 exploreP:0.7138
Episode:163 meanR:19.6100 R:34.0000 rate:0.0680 aloss:1.3985 dloss:0.4722 aloss2:5.1661 exploreP:0.7114
Episode:164 meanR:19.9200 R:41.0000 rate:0.0820 aloss:1.3946 dloss:0.4718 aloss2:5.1484 exploreP:0.7085
Episode:165 meanR:20.4900 R:77.0000 rate:0.1540 aloss:1.3929 dloss:0.4783 aloss2:5.1400 exploreP:0.7032
Episode:166 meanR:20.4200 R:28.0000 rate:0.0560 aloss:1.4002 dloss:0.5039 aloss2:5.2612 exploreP:0.7013
Episode:167 meanR:20.4000 R:19.0000 rate:0.0380 aloss:1.3986 dloss:0.5046 aloss2:5.4222 exploreP:0.6999
Episode:168 meanR:20.3100 R:18.0000 rate:0.0360 aloss:1.3926 dlo

Episode:238 meanR:24.0500 R:29.0000 rate:0.0580 aloss:1.3951 dloss:0.4624 aloss2:5.5220 exploreP:0.5906
Episode:239 meanR:23.9600 R:14.0000 rate:0.0280 aloss:1.3939 dloss:0.4662 aloss2:5.6495 exploreP:0.5898
Episode:240 meanR:23.9600 R:23.0000 rate:0.0460 aloss:1.3957 dloss:0.4978 aloss2:5.6175 exploreP:0.5885
Episode:241 meanR:23.9300 R:10.0000 rate:0.0200 aloss:1.3964 dloss:0.4323 aloss2:5.5293 exploreP:0.5879
Episode:242 meanR:23.8600 R:26.0000 rate:0.0520 aloss:1.4000 dloss:0.5042 aloss2:5.7781 exploreP:0.5864
Episode:243 meanR:23.9400 R:23.0000 rate:0.0460 aloss:1.4078 dloss:0.5023 aloss2:5.5962 exploreP:0.5851
Episode:244 meanR:23.7500 R:12.0000 rate:0.0240 aloss:1.3955 dloss:0.4629 aloss2:5.6044 exploreP:0.5844
Episode:245 meanR:23.8400 R:20.0000 rate:0.0400 aloss:1.4013 dloss:0.4761 aloss2:5.6331 exploreP:0.5832
Episode:246 meanR:23.8900 R:16.0000 rate:0.0320 aloss:1.3981 dloss:0.4859 aloss2:5.6222 exploreP:0.5823
Episode:247 meanR:23.9300 R:19.0000 rate:0.0380 aloss:1.3903 dlo

Episode:317 meanR:19.6600 R:11.0000 rate:0.0220 aloss:1.3952 dloss:0.4713 aloss2:5.5243 exploreP:0.5188
Episode:318 meanR:19.3800 R:14.0000 rate:0.0280 aloss:1.3906 dloss:0.4601 aloss2:5.5363 exploreP:0.5181
Episode:319 meanR:18.8300 R:9.0000 rate:0.0180 aloss:1.3849 dloss:0.4569 aloss2:5.5784 exploreP:0.5177
Episode:320 meanR:18.7700 R:10.0000 rate:0.0200 aloss:1.3915 dloss:0.4566 aloss2:5.6177 exploreP:0.5171
Episode:321 meanR:18.3400 R:15.0000 rate:0.0300 aloss:1.3953 dloss:0.4700 aloss2:5.5585 exploreP:0.5164
Episode:322 meanR:18.0900 R:13.0000 rate:0.0260 aloss:1.3931 dloss:0.4622 aloss2:5.5962 exploreP:0.5157
Episode:323 meanR:17.7100 R:11.0000 rate:0.0220 aloss:1.3883 dloss:0.4830 aloss2:5.5913 exploreP:0.5152
Episode:324 meanR:17.5900 R:10.0000 rate:0.0200 aloss:1.3884 dloss:0.4746 aloss2:5.5893 exploreP:0.5147
Episode:325 meanR:17.5200 R:8.0000 rate:0.0160 aloss:1.3960 dloss:0.4909 aloss2:5.5384 exploreP:0.5143
Episode:326 meanR:17.4200 R:8.0000 rate:0.0160 aloss:1.3884 dloss:

...

Episode:791 meanR:80.7000 R:38.0000 rate:0.0760 aloss:1.4368 dloss:0.4733 aloss2:5.2681 exploreP:0.1502
Episode:792 meanR:80.8800 R:31.0000 rate:0.0620 aloss:1.6666 dloss:0.6218 aloss2:5.0061 exploreP:0.1498
Episode:793 meanR:81.5600 R:79.0000 rate:0.1580 aloss:1.4934 dloss:0.5455 aloss2:5.1101 exploreP:0.1487
Episode:794 meanR:81.6900 R:22.0000 rate:0.0440 aloss:1.4022 dloss:0.5418 aloss2:5.2346 exploreP:0.1484
Episode:795 meanR:82.1100 R:55.0000 rate:0.1100 aloss:1.4617 dloss:0.4991 aloss2:5.2145 exploreP:0.1476
Episode:796 meanR:85.6800 R:376.0000 rate:0.7520 aloss:1.4519 dloss:0.5172 aloss2:5.3071 exploreP:0.1425
Episode:797 meanR:90.0500 R:448.0000 rate:0.8960 aloss:1.4215 dloss:0.5560 aloss2:5.2181 exploreP:0.1367
Episode:798 meanR:91.0100 R:109.0000 rate:0.2180 aloss:1.4287 dloss:0.5075 aloss2:5.1852 exploreP:0.1353
Episode:799 meanR:95.7000 R:500.0000 rate:1.0000 aloss:1.4358 dloss:0.5825 aloss2:5.0967 exploreP:0.1292
Episode:800 meanR:95.9100 R:39.0000 rate:0.0780 aloss:1.4465

Episode:870 meanR:153.8600 R:67.0000 rate:0.1340 aloss:1.4469 dloss:0.7861 aloss2:4.2603 exploreP:0.0434
Episode:871 meanR:153.9000 R:103.0000 rate:0.2060 aloss:1.4601 dloss:0.7415 aloss2:4.1913 exploreP:0.0430
Episode:872 meanR:154.1300 R:77.0000 rate:0.1540 aloss:1.4668 dloss:0.6766 aloss2:4.1805 exploreP:0.0428
Episode:873 meanR:154.3200 R:62.0000 rate:0.1240 aloss:1.5453 dloss:0.6395 aloss2:4.2157 exploreP:0.0426
Episode:874 meanR:154.2700 R:52.0000 rate:0.1040 aloss:1.5081 dloss:0.6825 aloss2:4.1964 exploreP:0.0424
Episode:875 meanR:154.1800 R:56.0000 rate:0.1120 aloss:1.5098 dloss:0.7410 aloss2:4.1473 exploreP:0.0422
Episode:876 meanR:154.2200 R:43.0000 rate:0.0860 aloss:1.5043 dloss:0.8177 aloss2:4.0722 exploreP:0.0421
Episode:877 meanR:154.3100 R:49.0000 rate:0.0980 aloss:1.5756 dloss:0.7004 aloss2:4.0994 exploreP:0.0419
Episode:878 meanR:154.2200 R:42.0000 rate:0.0840 aloss:1.5412 dloss:0.7742 aloss2:4.0969 exploreP:0.0418
Episode:879 meanR:154.2000 R:36.0000 rate:0.0720 aloss

Episode:948 meanR:142.7800 R:300.0000 rate:0.6000 aloss:1.3789 dloss:0.8767 aloss2:3.7841 exploreP:0.0210
Episode:949 meanR:142.8600 R:78.0000 rate:0.1560 aloss:1.3733 dloss:0.9598 aloss2:3.6348 exploreP:0.0209
Episode:950 meanR:143.2600 R:106.0000 rate:0.2120 aloss:1.4338 dloss:0.9070 aloss2:3.5679 exploreP:0.0208
Episode:951 meanR:142.8900 R:54.0000 rate:0.1080 aloss:1.4551 dloss:0.9821 aloss2:3.4870 exploreP:0.0207
Episode:952 meanR:142.4000 R:60.0000 rate:0.1200 aloss:1.4771 dloss:0.9145 aloss2:3.4798 exploreP:0.0207
Episode:953 meanR:142.1500 R:50.0000 rate:0.1000 aloss:1.4615 dloss:0.9907 aloss2:3.4550 exploreP:0.0206
Episode:954 meanR:139.5000 R:49.0000 rate:0.0980 aloss:1.4946 dloss:0.8993 aloss2:3.4866 exploreP:0.0206
Episode:955 meanR:135.1400 R:64.0000 rate:0.1280 aloss:1.4813 dloss:0.9196 aloss2:3.4957 exploreP:0.0205
Episode:956 meanR:134.5200 R:40.0000 rate:0.0800 aloss:1.4527 dloss:0.8789 aloss2:3.5162 exploreP:0.0204
Episode:957 meanR:133.9900 R:52.0000 rate:0.1040 alos

Episode:1026 meanR:251.2000 R:500.0000 rate:1.0000 aloss:1.3128 dloss:1.0916 aloss2:2.9029 exploreP:0.0115
Episode:1027 meanR:253.2600 R:388.0000 rate:0.7760 aloss:1.3326 dloss:1.0616 aloss2:2.8887 exploreP:0.0115
Episode:1028 meanR:257.2000 R:500.0000 rate:1.0000 aloss:1.3157 dloss:1.0890 aloss2:2.9407 exploreP:0.0114
Episode:1029 meanR:257.2000 R:500.0000 rate:1.0000 aloss:1.2994 dloss:1.0840 aloss2:2.9295 exploreP:0.0113
Episode:1030 meanR:259.4500 R:326.0000 rate:0.6520 aloss:1.2913 dloss:1.0882 aloss2:2.9288 exploreP:0.0113
Episode:1031 meanR:262.8800 R:500.0000 rate:1.0000 aloss:1.2966 dloss:1.1213 aloss2:2.8407 exploreP:0.0112
Episode:1032 meanR:267.1800 R:500.0000 rate:1.0000 aloss:1.3016 dloss:1.0884 aloss2:2.8964 exploreP:0.0112
Episode:1033 meanR:270.8500 R:500.0000 rate:1.0000 aloss:1.3278 dloss:1.0634 aloss2:2.9442 exploreP:0.0111
Episode:1034 meanR:270.8500 R:500.0000 rate:1.0000 aloss:1.2936 dloss:1.1059 aloss2:2.9060 exploreP:0.0110
Episode:1035 meanR:273.8200 R:500.000

Episode:1103 meanR:462.9900 R:500.0000 rate:1.0000 aloss:1.1496 dloss:1.3297 aloss2:1.8618 exploreP:0.0100
Episode:1104 meanR:466.8000 R:500.0000 rate:1.0000 aloss:1.1557 dloss:1.3058 aloss2:1.9016 exploreP:0.0100
Episode:1105 meanR:470.8600 R:500.0000 rate:1.0000 aloss:1.1365 dloss:1.3295 aloss2:1.8493 exploreP:0.0100
Episode:1106 meanR:475.1000 R:500.0000 rate:1.0000 aloss:1.1412 dloss:1.3154 aloss2:1.8651 exploreP:0.0100
Episode:1107 meanR:479.1000 R:500.0000 rate:1.0000 aloss:1.1381 dloss:1.3357 aloss2:1.8269 exploreP:0.0100
Episode:1108 meanR:479.1000 R:500.0000 rate:1.0000 aloss:1.1371 dloss:1.3252 aloss2:1.8129 exploreP:0.0100
Episode:1109 meanR:479.1000 R:500.0000 rate:1.0000 aloss:1.1362 dloss:1.3343 aloss2:1.7940 exploreP:0.0100
Episode:1110 meanR:479.1000 R:500.0000 rate:1.0000 aloss:1.1416 dloss:1.3176 aloss2:1.8295 exploreP:0.0100
Episode:1111 meanR:479.1000 R:500.0000 rate:1.0000 aloss:1.1373 dloss:1.3333 aloss2:1.7924 exploreP:0.0100
Episode:1112 meanR:479.1000 R:500.000

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')`

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.